In [243]:
exec(open("./fancy_string.py").read())

import pandas as pd
import numpy as np
import statistics

from string import punctuation
from re import sub

In [244]:
total_df = pd.read_csv('/Users/madke/Documents/AAPECSMadeline.csv', index_col = 0)

cats = total_df[pd.notnull(total_df.stressDescription)]
catu = total_df[pd.notnull(total_df.unpleasantEventDescription)] 
catu = catu[~(catu['unpleasantEventDescription'] == 'none')]
catu = catu[~(catu['unpleasantEventDescription'] == 'none ')]
catp = total_df[pd.notnull(total_df.pleasantEventDescription)]

stress_text = list(cats['stressDescription'])
unpleasant_text = list(catu['unpleasantEventDescription'])
pleasant_text = list(catp['pleasantEventDescription'])

IC_sample = stress_text + pleasant_text + unpleasant_text
pID = list(cats['participantID']) + list(catp['participantID']) + list(catu['participantID'])

translator = str.maketrans('','', sub('\#', '', punctuation))

IC_samplew = []
for i in IC_sample:
    n = i.split()
    sub = ""
    for z in n:
        z = z.lower()
        s = z.translate(str.maketrans(translator))
        sub = sub + " " + s
    IC_samplew.append(sub)

In [245]:
fancy_strs = [FancyString(i) for i in IC_samplew if i != 'none']

In [247]:
df_constructed = pd.DataFrame({
    
    'pID': pID,
    'passive': [i.is_passive() for i in fancy_strs],
    'wc': [i.wordcount() for i in fancy_strs],
    'mec': [i.me_count() for i in fancy_strs],
    'adc': [i.ad_count() for i in fancy_strs],
    'val': [1] * len(fancy_strs),
    'group': ['stress']*len(stress_text) +['pleasant']*len(pleasant_text)+['unpleasant']*len(unpleasant_text)
    
})

In [248]:
df_const_short = df_constructed.groupby(['pID', 'group']).agg('mean')

In [249]:
formal_vals = pd.pivot_table(df_const_short, index=['pID'], columns=['group'], aggfunc=np.sum, fill_value=0)
formal_vals = formal_vals.reset_index()
formal_vals.columns = ['pID', 'adc_stress', 'adc_pleasant', 'adc_unpleasant', 'mec_stress', 'mec_pleasant', 'mec_unpleasant', 'passive_stress', 'passive_pleasant',
       'passive_unpleasant', 'val_stress', 'val_pleasant', 'val_unpleasant', 'wc_stress', 'wc_pleasant', 'wc_unpleasant']
formal_vals.to_csv('/users/madke/documents/092520_aapecsformalvals.csv')

In [181]:
vector = [i.spacy_vec() for i in fancy_strs]

In [191]:
vec_df = pd.DataFrame(np.vstack(vector))
vec_df.to_csv('./092620_aapecsdims.csv')

In [250]:
print(len(stress_text)) 
print(len(pleasant_text))
print(len(unpleasant_text))

2446
4144
1016
